In [1]:
pip install -r requirements.txt -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.2/411.2 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.3/390.3 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.2/267.2 kB 18.4 MB/s eta 0:00:

In [ ]:
GROQ_API_KEY = "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"
QDRANT_URL = "https://efe5e4cc-d4c7-4289-93e9-300aa774fd91.us-east4-0.gcp.cloud.qdrant.io"
QDRANT_API_KEY =      "gBd-C2KJzAusieQXl-wuqTk_XgZ2ZXz6t8wQ2q2uy7qBKjRnV2sJZg"
QDRANT_COLLECTION_NAME = "product_list_AI_Training"
model_name = "paraphrase-multilingual-MiniLM-L12-v2"
query_text = "Please advise me on suitable foods for health-conscious individuals. Advise me foods have rich nutritional profile. Please guide me through the payment process. Give me an excellent breakfast option"

# Read data from Docx file and PDF file

In [71]:
import os
import json
import docx
import fitz  # PyMuPDF for PDF processing
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document


# Function to extract text from DOCX
def extract_text_from_docx(docx_path):
    doc = docx.Document(docx_path)
    text = []
    for para in doc.paragraphs:
        text.append(para.text)
    return "\n".join(text)


# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = []
    for page in doc:
        text.append(page.get_text())
    return "\n".join(text)


# Function to identify file type and process it
def process_file(file_path):
    file_extension = file_path.split('.')[-1].lower()  # Get file extension

    # Identify the file type and extract text accordingly
    if file_extension == 'docx':
        return extract_text_from_docx(file_path)
    elif file_extension == 'pdf':
        return extract_text_from_pdf(file_path)
    else:
        raise ValueError(f"Unsupported file type: {file_extension}")


# Function to save split documents with metadata into JSON format
def save_data_locally_with_metadata(documents, filename, directory, source_file, language="en"):
    # Create directory if it does not exist
    if not os.path.exists(directory):
        os.makedirs(directory)

    file_path = os.path.join(directory, filename)  # Full file path

    # Add metadata and prepare data to save
    data_to_save = []
    for doc in documents:
        # Add metadata fields
        metadata = {
            "source": source_file,  # Original file name
            "content_type": "text",  # Content type (text for now)
            "title": doc.page_content[:100].strip() + "...",  # Title: First 100 characters as a summary
            "language": language,  # Language (manually set or detected)
        }
        data_to_save.append({"page_content": doc.page_content, "metadata": metadata})

    # Save data to JSON file
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(data_to_save, file, ensure_ascii=False, indent=4)
    print(f'Data saved to {file_path}')  # Notify success


# Main function
if __name__ == "__main__":
    file_path = "/content/Product List AI Training.docx"  # Change this to your file path
    output_dir = "./data"  # Directory to save JSON files
    output_filename = "Product List AI Training.json"  # JSON file name

    # Extract text
    extracted_text = process_file(file_path)

    # Split text
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=5000, chunk_overlap=500)
    splits = text_splitter.split_documents([Document(page_content=extracted_text)])

    # Save splits in the desired JSON format with metadata
    save_data_locally_with_metadata(splits, output_filename, output_dir, source_file=os.path.basename(file_path))

    # Print the number of splits
    print(f"Number of splits: {len(splits)}")


Data saved to ./data/Product List AI Training.json
Number of splits: 44


# Crawl data from website

In [68]:
import os
import re
import json
from langchain_community.document_loaders import RecursiveUrlLoader, WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from bs4 import BeautifulSoup
from dotenv import load_dotenv

# load_dotenv()
def bs4_extractor(html: str) -> str:
    soup = BeautifulSoup(html, "html.parser")  # Phân tích cú pháp HTML
    return re.sub(r"\n\n+", "\n\n", soup.text).strip()  # Xóa khoảng trắng và dòng trống thừa

def crawl_web(url_data):

    # Tạo loader với độ sâu tối đa là 4 cấp
    loader = RecursiveUrlLoader(url=url_data, extractor=bs4_extractor, max_depth=4)
    docs = loader.load()  # Tải nội dung
    print('length: ', len(docs))  # In số lượng tài liệu đã tải

    # Chia nhỏ văn bản thành các đoạn 10000 ký tự, với 500 ký tự chồng lấp
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=5000, chunk_overlap=500)
    all_splits = text_splitter.split_documents(docs)
    print('length_all_splits: ', len(all_splits))  # In số lượng đoạn văn bản sau khi chia
    return all_splits

def web_base_loader(url_data):
    loader = WebBaseLoader(url_data)  # Tạo loader cơ bản
    docs = loader.load()  # Tải nội dung
    print('length: ', len(docs))  # In số lượng tài liệu

    # Chia nhỏ văn bản tương tự như trên
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
    all_splits = text_splitter.split_documents(docs)
    return all_splits

def save_data_locally(documents, filename, directory):
    # Tạo thư mục nếu chưa tồn tại
    if not os.path.exists(directory):
        os.makedirs(directory)

    file_path = os.path.join(directory, filename)  # Tạo đường dẫn đầy đủ

    # Chuyển đổi documents thành định dạng có thể serialize
    data_to_save = [{'page_content': doc.page_content, 'metadata': doc.metadata} for doc in documents]
    # Lưu vào file JSON
    with open(file_path, 'w') as file:
        json.dump(data_to_save, file, indent=4)
    print(f'Data saved to {file_path}')  # In thông báo lưu thành công

def main():
    WEB_URL = "https://en.wikipedia.org/wiki/Nikola_Tesla"
    # Crawl dữ liệu từ trang docs của stack-ai
    data = crawl_web(WEB_URL)
    # Lưu dữ liệu vào thư mục data_v2
    save_data_locally(data, 'stack.json', 'data')
    print('data: ', data)  # In dữ liệu đã crawl

# Kiểm tra nếu file được chạy trực tiếp
if __name__ == "__main__":
    main()

length:  10
length_all_splits:  66
Data saved to data/stack.json
data:  [Document(metadata={'source': 'https://en.wikipedia.org/wiki/Nikola_Tesla', 'content_type': 'text/html; charset=UTF-8', 'title': 'Nikola Tesla - Wikipedia', 'language': 'en'}, page_content="Nikola Tesla - Wikipedia\n\nJump to content\n\nMain menu\n\nMain menu\nmove to sidebar\nhide\n\n\t\tNavigation\n\t\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact us\n\n\t\tContribute\n\t\n\nHelpLearn to editCommunity portalRecent changesUpload file\n\nSearch\n\nSearch\n\nAppearance\n\nDonate\n\nCreate account\n\nLog in\n\nPersonal tools\n\nDonate Create account Log in\n\n\t\tPages for logged out editors learn more\n\nContributionsTalk\n\nContents\nmove to sidebar\nhide\n\n(Top)\n\n1\nEarly years\n\nToggle Early years subsection\n\n1.1\nWorking at Budapest Telephone Exchange\n\n2\nWorking at Edison\n\nToggle Working at Edison subsection\n\n2.1\nMoving to the United States\n\n3\nTesla Electric Light and Man

# Vector store

In [72]:
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.http.models import VectorParams, Distance
from qdrant_client.http.models import PointStruct
import json

def load_sentence_transformer(model_name: str):
    model = SentenceTransformer(model_name)
    return model

def init_qdrant_client(qdrant_url: str, qdrant_api_key: str):
    client = QdrantClient(url=qdrant_url, api_key=qdrant_api_key)
    return client

def upload_to_qdrant(qdrant_client, collection_name: str, model, documents: list):
    dimension = model.get_sentence_embedding_dimension()

    # Create collection in Qdrant if not exists
    qdrant_client.recreate_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=dimension, distance=Distance.COSINE)
    )

    # Generate embeddings for the documents
    embeddings = model.encode(documents, show_progress_bar=True)

    # Prepare points for uploading to Qdrant
    points = [
        PointStruct(id=i, vector=embedding.tolist(), payload={"text": documents[i]})
        for i, embedding in enumerate(embeddings)
    ]

    # Upload points to Qdrant
    qdrant_client.upsert(
        collection_name=collection_name,
        points=points,
    )

    print(f"Uploaded {len(points)} documents to Qdrant!")

def create_vectorstore_and_upload(qdrant_url: str, qdrant_api_key: str, collection_name: str, model_name: str, documents_file: str):
    model = load_sentence_transformer(model_name)

    qdrant_client = init_qdrant_client(qdrant_url, qdrant_api_key)

    with open(documents_file, "r") as file:
        documents = json.load(file)

    upload_to_qdrant(qdrant_client, collection_name, model, documents)
    return qdrant_client

## Stored Documents

In [ ]:
documents_file = "/content/data/Product List AI Training.json"

In [74]:
# Gọi hàm tổng hợp
qdrant_vectorstore = create_vectorstore_and_upload(QDRANT_URL, QDRANT_API_KEY, QDRANT_COLLECTION_NAME, model_name, documents_file)


<ipython-input-72-65fee38ae7dd>:19: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant_client.recreate_collection(


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Uploaded 44 documents to Qdrant!


## Test vectorstore

In [90]:
embedding_model = load_sentence_transformer(model_name)
query_text = "Please advise me on suitable foods for health-conscious individuals. Advise me foods have rich nutritional profile. Please guide me through the payment process. Give me an excellent breakfast option"
query_embedding = embedding_model.encode(query_text)

results = qdrant_vectorstore.search(
    collection_name=QDRANT_COLLECTION_NAME,
    query_vector=query_embedding.tolist(),
)

In [91]:
print("Search results:")
for result in results:
    print(f"{result}")
    break
    print(f"Text: {result.payload['text']}, \nScore: {result.score}")

Search results:
id=32 version=0 score=0.43434024 payload={'text': {'page_content': 'Product: Aashirvaad Daliya\nAashirvaad Daliya is another staple that deserves a place in every health-conscious kitchen. Made from premium quality wheat, Aashirvaad Daliya is crafted to deliver both taste and nutrition, ensuring that your meals are wholesome and fulfilling.\nNutritional Value: Packed with dietary fiber, protein, and essential vitamins and minerals, Aashirvaad Daliya offers a balanced nutritional profile. It helps maintain digestive health by promoting regular bowel movements and fostering a healthy gut microbiome. The high fiber content ensures sustained energy release, keeping you full and energized throughout the day. Additionally, the protein in daliya supports muscle repair and growth, making it an excellent choice for those leading an active lifestyle. Vitamins such as B-complex and minerals like magnesium and iron contribute to overall well-being by enhancing metabolic processes a

# Model

## test model LLM

In [92]:
import os
from groq import Groq

client = Groq(
    api_key=GROQ_API_KEY,
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": query_text,
        }
    ],
    model="llama3-8b-8192",
)

In [93]:
print(chat_completion.choices[0].message.content)

I'd be happy to help!

**Nutrient-Dense Foods:**

Here are some healthy food options that are rich in nutrients and perfect for health-conscious individuals:

1. Leafy Greens: Spinach, kale, collard greens, and more
	* Rich in vitamins A, C, and K, calcium, and iron
2. Berries: Blueberries, raspberries, strawberries, and more
	* Packed with antioxidants, vitamins C and K, and fiber
3. Fatty Fish: Salmon, sardines, tuna, and mackerel
	* High in omega-3 fatty acids, protein, and vitamin D
4. Sweet Potatoes
	* Rich in vitamin A, fiber, and potassium
5. Avocados
	* Good source of healthy fats, fiber, and various vitamins and minerals
6. Legumes: Lentils, chickpeas, black beans, and more
	* High in protein, fiber, and various vitamins and minerals
7. Whole Grains: Brown rice, quinoa, whole wheat, and more
	* Rich in fiber, iron, and B vitamins
8. Nuts and Seeds: Almonds, walnuts, chia seeds, and more
	* Good source of healthy fats, protein, and fiber

**Breakfast Option:**

Here's an excell

## querry for retrieving vectorstore  

In [83]:
query_text = "Please advise me on suitable foods for health-conscious individuals. Advise me foods have rich nutritional profile. Please guide me through the payment process. Give me an excellent breakfast option"

In [84]:
from qdrant_client.models import SearchRequest
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
from qdrant_client import QdrantClient
from qdrant_client.http.models import VectorParams, Distance
from qdrant_client.http.models import PointStruct
import json
from langchain.vectorstores import FAISS
from langchain_community.retrievers import BM25Retriever
from langchain.schema import Document
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

top_k = 5

def load_sentence_transformer(model_name: str):
    model = SentenceTransformer(model_name)
    return model

def init_qdrant_client(qdrant_url: str, qdrant_api_key: str):
    client = QdrantClient(url=qdrant_url, api_key=qdrant_api_key)
    return client

embedding_model = load_sentence_transformer(model_name)
query_embedding = embedding_model.encode(query_text)
qdrant_vectorstore = init_qdrant_client(QDRANT_URL, QDRANT_API_KEY)

results = qdrant_vectorstore.search(
    collection_name=QDRANT_COLLECTION_NAME,
    query_vector=embedding_model.encode(query_text).tolist(),
)

# Parse results into Document objects
documents = []
for result in results:
    if 'text' in result.payload and 'page_content' in result.payload['text']:
        page_content = result.payload['text']['page_content']
        metadata = result.payload['text'].get('metadata', {})
        documents.append(Document(page_content=page_content, metadata=metadata))


## test hybrid (ensemble retriever)  
made from BM25-sparse retriever and - MiniLM-L12-v2dense retriever

In [85]:
from langchain.retrievers import EnsembleRetriever

# Create BM25 retriever using the Document objects
bm25_retriever = BM25Retriever.from_documents(documents)
bm25_retriever.k = top_k

# Encode document texts for FAISS
doc_texts = [doc.page_content for doc in documents]
embeddings = embedding_model.encode(doc_texts)
text_embeddings = list(zip(doc_texts, embeddings))

# Create the FAISS retriever
faiss_index = FAISS.from_documents(documents, HuggingFaceEmbeddings(model_name=model_name))
retriever = faiss_index.as_retriever(search_kwargs={"k": top_k})

# Combine BM25 and FAISS retrievers in an EnsembleRetriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[retriever, bm25_retriever],
    weights=[0.7, 0.3],
)

# Query the ensemble retriever
relevant_docs = ensemble_retriever.get_relevant_documents(query_text)

# Output the retrieved documents
print("Retrieved documents:")
for i, doc in enumerate(relevant_docs):
    print(f"Document {i + 1}: {doc.page_content}")
    print("-" * 80)

Retrieved documents:
Document 1: Product: Aashirvaad Daliya
Aashirvaad Daliya is another staple that deserves a place in every health-conscious kitchen. Made from premium quality wheat, Aashirvaad Daliya is crafted to deliver both taste and nutrition, ensuring that your meals are wholesome and fulfilling.
Nutritional Value: Packed with dietary fiber, protein, and essential vitamins and minerals, Aashirvaad Daliya offers a balanced nutritional profile. It helps maintain digestive health by promoting regular bowel movements and fostering a healthy gut microbiome. The high fiber content ensures sustained energy release, keeping you full and energized throughout the day. Additionally, the protein in daliya supports muscle repair and growth, making it an excellent choice for those leading an active lifestyle. Vitamins such as B-complex and minerals like magnesium and iron contribute to overall well-being by enhancing metabolic processes and maintaining energy levels.
Recommended For: Aashir

## test each retriever

In [89]:
# Query for both retrievers separately and print results

print("=== BM25 Retriever Results ===")
bm25_results = bm25_retriever.get_relevant_documents(query_text)
for i, doc in enumerate(bm25_results):
    print(f"Document {i + 1}: {doc.page_content}")
    print("-" * 80)

print("\n=== FAISS (MiniLM-L12-v2) Retriever Results ===")
faiss_results = retriever.get_relevant_documents(query_text)
for i, doc in enumerate(faiss_results):
    print(f"Document {i + 1}: {doc.page_content}")
    print("-" * 80)

=== BM25 Retriever Results ===
Document 1: Product: Patanjali Poha
A Wholesome and Nutritious Meal for Every Lifestyle
Patanjali Poha is a premium offering crafted from the finest quality rice, delivering both nutrition and taste in every bite. Known for its high quality and commitment to purity, Patanjali Poha is an excellent choice for those seeking healthy, convenient, and versatile meal options. This poha, made under the trusted Patanjali brand, combines the richness of traditional Indian food with the convenience demanded by modern lifestyles, making it a favorite for individuals and families across the globe.
Nutritional Value
The nutritional profile of Patanjali Poha makes it a standout choice for health-conscious individuals. As a rich source of carbohydrates, it provides the energy required to power through busy days. Unlike simple carbohydrates, which lead to rapid spikes and crashes in energy levels, the complex carbohydrates in Patanjali Poha ensure sustained energy release

## test pipeline RAG + LLM

In [88]:
# Get relevant documents from the ensemble retriever
relevant_docs = ensemble_retriever.get_relevant_documents(query_text)

# Format retrieved documents into a prompt
prompt = f"Query: {query_text}\n\nRelevant Documents:\n"
for i, doc in enumerate(relevant_docs):
    prompt += f"Document {i + 1}:\n{doc.page_content}\n\n"

# Call Groq LLM with the formatted prompt
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="llama3-8b-8192",
)

# Output the generated response from the model
print("Response from Groq LLM:")
print(chat_completion.choices[0].message.content)


Response from Groq LLM:
Based on the documents provided, here are some suitable foods for health-conscious individuals:

1. **Aashirvaad Daliya**: A nutritious and filling breakfast option, suitable for individuals of all age groups, including those with digestive issues or food allergies.
2. **Organic India Mixed Grains**: A blend of nutrient-dense grains, ideal for health-conscious individuals and families looking for a wholesome breakfast option.
3. **Patanjali Poha**: A versatile and nutritious breakfast option, suitable for individuals with diverse dietary needs and preferences.
4. **Tata Sampann Thick Poha**: A premium quality poha suitable for adults and children, with no artificial additives or preservatives.
5. **Lundberg Family Farms Organic Long-Grain Brown Rice**: A nutritious and filling side dish option, suitable for individuals with heart health concerns, digestive issues, or those looking to manage their weight.
6. **Thrive Market Organic Medium Black Rice**: A nutrient

In [ ]:
D